# Tender2Project
The following Kaggle notebook exploits the long context window of Gemini, in order to fulfill the following targets:
- Analyze the tender for a project.
- Analyze the information about possible products, provided from different companies.
- Find the best combination of products to build the project in the tender, identifying the most compliant company as well.

## Notebook structure
The notebook is composed by different parts, each one with a specific target:
- A tender for a project is parsed, such that its information is converted to text.
- Information scraped from the websites of different companies is loaded as text.
- All the text is forwarded to Gemini, whereas a system prompt and a user prompt are written to explain the purposed o Gemini.

## Theoretical aspects
The current way to use Gemini makes use of the following properties of a LLM (Large Language Model) like Gemini:
|  **LLM property** | **Where it is used** | **How it is used** |
|:-----------------:|:--------------------:|:------------------:|
|     Reasoning     |          TBD         |         TBD        |
|       Memory      |          TBD         |         TBD        |
| Chain of Thoughts |          TBD         |         TBD        |
|                   |                      |                    |

Clean the working directory

In [49]:
! rm -r /kaggle/working/*

In [50]:
from IPython.display import Markdown

# Analyze the tenders

Locate the tenders - in PDF format.

In [51]:
# fetch the script to download content from GitHub
!wget https://raw.githubusercontent.com/gabripo/kaggle-gemini-long-context/refs/heads/main/github_downloader.py -P /kaggle/working/scripts

# add downloaded script to the Python path
import sys
sys.path.append('/kaggle/working/scripts')

--2024-11-22 23:13:08--  https://raw.githubusercontent.com/gabripo/kaggle-gemini-long-context/refs/heads/main/github_downloader.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1722 (1.7K) [text/plain]
Saving to: '/kaggle/working/scripts/github_downloader.py'

github_downloader.p 100%[===================>]   1.68K  --.-KB/s    in 0s      

2024-11-22 23:13:08 (28.6 MB/s) - '/kaggle/working/scripts/github_downloader.py' saved [1722/1722]



In [52]:
import github_downloader

github_downloader.download_files_from_github_repo(folderName="tenders", saveFolder="/kaggle/working/tenders", extension="pdf")

/kaggle/working/tenders/tender_solar.pdf downloaded successfully.
/kaggle/working/tenders/tender_wind.pdf downloaded successfully.
All files downloaded.


In [53]:
import os

# if the PDF file is given as Kaggle Input (for example, manually uploaded), change the use_kaggle_input to True
use_kaggle_input_tender = False if os.path.exists('/kaggle/working/tenders') else True
if use_kaggle_input_tender:
    tenders_file_path = '/kaggle/input/tenders'
else:
    tenders_file_path = '/kaggle/working/tenders'

print(f"The folder {tenders_file_path} will be considered as containing the tenders")

The folder /kaggle/working/tenders will be considered as containing the tenders


Installing required Python packages to analyze the tender - in PDF format.

In [54]:
!pip install PyPDF2

Extract information from the tenders.
The output will be a text.

In [55]:
import os
from PyPDF2 import PdfReader

tenders = [t for t in os.listdir(tenders_file_path) if t.endswith(".pdf")]
tenders_info = {}
for tender in tenders:
    print(f"Reading the tender {tender} ...")
    reader = PdfReader(os.path.join(tenders_file_path, tender))

    tenders_info[tender] = {}
    tenders_info[tender]["name"] = tender
    tenders_info[tender]["content"] = "\n".join([page.extract_text() for page in reader.pages])

# information for each tender can be accessed by:
# tenders_info[tenders[0]]

Reading the tender tender_solar.pdf ...
Reading the tender tender_wind.pdf ...


# Fetch information about companies

## Overview
Information about interesting companies is obtained from their websites.

To generate data out of the companies' websites, we implemented a crawler.
The final output of the crawler is a JSON file, in which each field refers to a company: for each company, all the information of the websites is merged.

> To make things easier, the mentioned JSON file will be fetched from a Git repository where the crawling function has already been executed.

## Details about the crawling process:
- **Recursive scan**: after a webpage is scanned and its content is stored, eventual found sublinks are scanned, as well. A limit of the wepages to download is given as input.
- **Redundant information is deleted**: if some website content can be found multiple times in all the webpages of one company, then it is skipped. *Example*: undesired and redundant lines like "Contact Us" are removed, ensuring that the final content does not include unnecessary sentences.
- **Caching of already downloaded pages**: for each webpage, the content is stored in a JSON file, as well as the found sublinks. *Example*: after a run with a limit of N pages, other runs with less than N pages will use the stored files instead downloading data from internet; at the contrary, if the limit is increased to M > N pages, only M - N additional pages will be downloaded while the first N pages will be taken from the stored file.

## Load the results from the crawler's repo

In [56]:
github_downloader.download_files_from_github_repo(folderName="", saveFolder="/kaggle/working/companies_info", extension="json")

/kaggle/working/companies_info/companies_info.json downloaded successfully.
All files downloaded.


Locate the JSON file containing the companies' information.

In [57]:
import os

# if the JSON file is given as Kaggle Input (for example, manually uploaded), change the use_kaggle_input to True
use_kaggle_input_companies = False if os.path.exists('/kaggle/working/companies_info') else True
companies_json_name = 'companies_info.json'
if use_kaggle_input_companies:
    companies_info_file_path = os.path.join('/kaggle/input/companies-info', companies_json_name)
else:
    companies_info_file_path = os.path.join('/kaggle/working/companies_info', companies_json_name)

print(f"The file {companies_info_file_path} will be used for the information regarding the companies")

The file /kaggle/working/companies_info/companies_info.json will be used for the information regarding the companies


Define a small function to read the information about the companies - in JSON format.

In [58]:
import json

def read_companies_info(jsonFilePath: str) -> dict:
    if os.path.exists(jsonFilePath):
        with open(jsonFilePath, "r") as f:
            data = json.load(f)
        return data
    else:
        return {}

Load the companies' information by using the defined function.

In [59]:
companies_info = read_companies_info(companies_info_file_path)

# companies_info is a dictionary, where the key is the name of the company and the related value its information
# print(companies_info["SIEMENS"]) 

# Chat with Gemini

In [60]:
# API key got here: https://ai.google.dev/tutorials/setup

import google.generativeai as genai
from kaggle_secrets import UserSecretsClient


user_secrets = UserSecretsClient()
secret_key = user_secrets.get_secret("GEMINI_API_KEY")

genai.configure(api_key = secret_key)

model_name = 'gemini-1.5-flash-latest'
model = genai.GenerativeModel(model_name=model_name)

chat = model.start_chat()

model_info = genai.get_model(f"models/{model_name}")
print(f"{model_info.input_token_limit=}")
print(f"{model_info.output_token_limit=}")

model_info.input_token_limit=1000000
model_info.output_token_limit=8192


## Define the system and user prompts

In [61]:
system_prompt = "You are an experienced technical sales manager. You are given the full portfolio for products and solutions of companies, through their website. Use this information to provide detailed technical answers based on the tender requirements which were given to you."

Markdown(system_prompt)

You are an experienced technical sales manager. You are given the full portfolio for products and solutions of companies, through their website. Use this information to provide detailed technical answers based on the tender requirements which were given to you.

In [62]:
tender_prompt_template = "The document you have is a tender, that contains technical requirements for a project. Summarize the technical requirements. The content of the document is: "
tender_prompts = []
for info in tenders_info.values():
    tender_prompts.append(f"You have a document called {info['name']} . " + tender_prompt_template + f"{info['content']}")

In [63]:
user_prompt = f"""

1. Identify all the technical requirements in the tenders' information you have.

2. For company [SIEMENS] and [HITACHI], find the respective relevant products and solutions with respect to point 1. Do this one company at a time and store the results.

3. Calculate an affinity score in percentage of company [SIEMENS] and [HITACHI] based on the match of results in point 2. Explain the way how you computed this percentage.

SIEMENS: {companies_info["SIEMENS"]}
HITACHI: {companies_info["HITACHI"]}

4. Return a quick technical summary of the tender.

5. Return the technical details of the company with the highest affinity score and its score. Focus on the technical specifications mentioning the compliances with the tender. Report also the URL of the source where you found the informations and mention if there is some non compliant requirements from tender.

6. Return also the other affinity scores alone of the other companies. 

"""

# Markdown(user_prompt)

## Analyze all the tenders

In [64]:
from time import sleep

want_a_response = True # disable if willing to save a notebook but the Gemini quota has been exhausted

responses = []
if want_a_response:
    num_queries = 0
    for tender_prompt, tender_name in zip(tender_prompts, tenders):
        print(f"Generating response for tender {tender_name} ...")
        response = chat.send_message(tender_prompt)
        # print(response.text)
        responses.append((tender_prompt, response.text, response.usage_metadata))
        print(f"Response for tender {tender_name} generated.")

        num_queries += 1
        """
        if num_queries % 2 == 0:
            wait_time_seconds = 90
            print(f"Waiting {wait_time_seconds} before continuing, to not exceed Gemini's quota")
            sleep(wait_time_seconds)
        """

    print("Analysis of the tenders concluded!")

Generating response for tender tender_solar.pdf ...
Response for tender tender_solar.pdf generated.
Generating response for tender tender_wind.pdf ...
Response for tender tender_wind.pdf generated.
Analysis of the tenders concluded!


In [65]:
# example how to include the chat history here https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/getting-started/intro_gemini_chat.ipynb
# description of the Content class here https://github.com/google-gemini/generative-ai-python/blob/main/docs/api/google/generativeai/GenerativeModel.md
from google.generativeai.protos import Content, Part

history_chat = []
for idx_response, response in enumerate(responses):
    query = Part()
    query.text = response[0]
    # TODO: consider different users? Example: for the tender number idx_response, use f"user_{idx_response}"
    history_chat.append(Content(role="user", parts=[query]))

    answer = Part()
    answer.text = response[1]
    history_chat.append(Content(role="model", parts=[answer]))

## Test the history

In [66]:
chat_with_memory_temp = model.start_chat(history=history_chat)
response_temp = chat_with_memory_temp.send_message("What are the tenders you have about?")

Markdown(response_temp.text)

I have access to information from two tender documents: one for a solar power plant project and one concerning wind energy projects.

The **solar power plant tender** detailed technical specifications for the construction of a 200 kWp solar PV plant, including a mini-grid.  This encompassed everything from the solar panels themselves and their mounting structures to the inverters, batteries, diesel generator backup, electrical cabling and infrastructure,  monitoring systems, building construction, and after-sales service.  The tender clearly outlined required specifications and standards for each component.

The **wind energy tender** document was more of a policy analysis and review of different tendering schemes for wind energy projects.  It did not present a single project tender but instead discussed various approaches to designing tenders for both onshore and offshore wind farms across different countries. The focus was on analyzing successful and unsuccessful elements of past tender designs, including different price-setting mechanisms, pre-qualification criteria, penalty systems, and the overall impact on project delivery rates and costs.  It offered a comparative analysis of best practices and pitfalls in various national tender programs.


## Count the needed tokens

In [67]:
print(f"{model.count_tokens(history_chat)=}")
print(f"{model.count_tokens(system_prompt + user_prompt)=}")

model.count_tokens(history_chat)=total_tokens: 20606

model.count_tokens(system_prompt + user_prompt)=total_tokens: 645265



## Generate a response with chat history

In [69]:
if want_a_response:
    chat_with_memory = model.start_chat(history=history_chat)
    
    print("Providing prompts based on the previous analysis of the tenders ...")
    response = chat_with_memory.send_message(system_prompt + user_prompt)
    # print(response.text)
    responses.append((system_prompt + user_prompt, response.text, response.usage_metadata))

    print("Response to the prompts is ready!")

Providing prompts based on the previous analysis of the tenders ...


ResourceExhausted: 429 Resource has been exhausted (e.g. check quota).

## Check how many tokens have been used so far

In [ ]:
if want_a_response:
    tot_tokens = 0
    for idx_response, response in enumerate(responses):
        response_token = response[2].total_token_count
        print(f"Token count for response {idx_response}: {response_token}")
        tot_tokens += response_token

    print(f"Total number of tokens: {tot_tokens}")

In [ ]:
if responses:
    print(responses[-1][1])